In [1]:
import warnings
warnings.filterwarnings('ignore')

# importing libaries


In [3]:
from datasets import load_dataset
import pandas as pd
from pprint import pprint

# loading dataset

In [4]:
dataset = load_dataset('craigslist_bargains', split="train")

Using custom data configuration default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5247 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/838 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/597 [00:00<?, ? examples/s]

Dataset craigslist_bargains downloaded and prepared to /root/.cache/huggingface/datasets/craigslist_bargains/default/1.1.0/9e4e64d13ea36e435a8f56f9305d3c771115909b7fe83269bf7cd8dd40302338. Subsequent calls will reuse this data.


# creating prompt 

In [5]:
dataset[0]

{'agent_info': {'Bottomline': ['None', 'None'],
  'Role': ['buyer', 'seller'],
  'Target': [7.0, 10.0]},
 'agent_turn': [0, 1, 0, 1, 0, 1, 0, 0, 1],
 'dialogue_acts': {'intent': ['init-price',
   'unknown',
   'insist',
   'counter-price',
   'counter-price',
   'counter-price',
   'agree',
   'offer',
   'accept'],
  'price': [5.0, -1.0, 5.0, 8.0, 6.0, 7.0, 7.0, 7.0, -1.0]},
 'utterance': ['Hi, not sure if the charger would work for my car. Can you sell it to me for $5?',
  'It will work, i have never seen a car without a cigarette lighter port.\\',
  "Still, can I buy it for $5? I'm on a tight budge",
  'I think the lowest I would want to go is 8. ',
  "How about $6 and I pick it up myself? It'll save you shipping to me.",
  '7, and we have a deal.',
  'Eh, fine. $7.',
  '',
  ''],
 'items': {'Category': ['phone', 'phone'],
  'Images': ['phone/6149527852_0.jpg', 'phone/6149527852_0.jpg'],
  'Price': [10.0, 10.0],
  'Description': ['Charge two devices simultaneously on the go. This ve

In [6]:
def create_prompt_dict(data):
    # Extract data
    agent_info = data['agent_info']
    agent_turn = data['agent_turn']
    dialogue_acts = data['dialogue_acts']
    utterances = data['utterance']
    items = data['items']



    # Additional information
    agent_role = ['buyer' if turn == 0 else 'seller' for turn in agent_turn]
    agent_target = agent_info['Target']
    dialogue_intent = dialogue_acts['intent']
    dialogue_price = dialogue_acts['price']
    item_category = items['Category'][0]
    item_images = items['Images']
    item_price = items['Price'][0]
    item_description = items['Description']
    item_title = items['Title']

    # Construct the full prompt in dictionary format
    full_prompt_dict = {
        "Title": item_title,
        "Category": item_category,
        "Price": item_price,
        "Target": agent_target,
        "Description": item_description,
        "Turn": agent_turn,
        "Conversation": utterances,
        

    }

    return full_prompt_dict



In [7]:
formatted_prompts = list(map(create_prompt_dict, dataset))

In [8]:
pprint(formatted_prompts[5])


{'Category': 'furniture',
 'Conversation': ['Hi, $15 sound good?',
                  'Can you come down a little more?  The condition is only '
                  'average.',
                  'Alright, $14?',
                  'Actually I was looking for something closer to $10.',
                  'Sounds good. $10 is fine.',
                  '',
                  ''],
 'Description': ['Still sturdy and good, average condition. Some small tear on '
                 'some by being placed against wall, but doesnt show much',
                 'Still sturdy and good, average condition. Some small tear on '
                 'some by being placed against wall, but doesnt show '
                 'much,Great For waiting room, client chairs etc.'],
 'Price': 18.0,
 'Target': [10.0, 18.0],
 'Title': ['Office Client Waiting Room CHAIRS',
           'Office Client Waiting Room CHAIRS'],
 'Turn': [1, 0, 1, 0, 1, 1, 0]}


In [9]:
dataframe  = pd.DataFrame(formatted_prompts)
dataframe.head()

,Title,Category,Price,Target,Description,Turn,Conversation
0,[Verizon Car Charger with Dual Output Micro US...,phone,10.0,"[7.0, 10.0]",[Charge two devices simultaneously on the go. ...,"[0, 1, 0, 1, 0, 1, 0, 0, 1]","[Hi, not sure if the charger would work for my..."
1,"[Long board custom bilt from the board up, Lon...",bike,200.0,"[120.0, 200.0]",[Hey there cl I have a long board for sale pre...,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1]","[Hi,I am interested in your board!, Hi, do you..."
2,[A 4 bedroom single family house for rent in c...,housing,3200.0,"[1920.0, 3200.0]",[This is a single family house in an excellent...,"[0, 1, 0, 1, 0, 1, 1, 0, 1, 0]","[i would love to buy , Sure! What's your price..."
3,"[Sectional Couch, Sectional Couch]",furniture,585.0,"[444.0, 585.0]",[I am remodeling my space and selling a beauti...,"[0, 1, 0, 1, 0, 1]","[hello, Hi, How old is the couch?, It is pract..."
4,"[Beautiful Cherry Hutch, Beautiful Cherry Hutch]",furniture,1000.0,"[600.0, 1000.0]","[MUST SEE!!!,Like new ! - making space in our ...","[0, 1, 1, 1]","[500, that is a lot lower then i am willing to..."


In [10]:
dataframe['Conversation'][0][0]

'Hi, not sure if the charger would work for my car. Can you sell it to me for $5?'

In [11]:
def group_and_concatenate(row):
    grouped_binary = []
    grouped_sentences = []

    current_binary = None
    current_sentences = []

    for binary, sentence in zip(row['Turn'], row['Conversation']):
        if current_binary is None:
            current_binary = binary
            current_sentences.append(sentence)
        elif current_binary == binary:
            current_sentences.append(sentence)
        else:
            grouped_binary.append(current_binary)
            grouped_sentences.append(" ".join(current_sentences))
            current_binary = binary
            current_sentences = [sentence]

    # Append the last group if it exists
    if current_binary is not None:
        grouped_binary.append(current_binary)
        grouped_sentences.append(" ".join(current_sentences))

    return pd.Series({'Grouped_Binary': grouped_binary, 'Concatenated_Sentences': grouped_sentences})

# Sample data
binary_list = [1, 0, 1, 1, 0, 1]
sentences_list = ["Sentence 1", "Sentence 2", "Sentence 3", "Sentence 4", "Sentence 5", "Sentence 6"]



In [12]:
dataframe[['Turn', 'Conversation']] = dataframe.apply(group_and_concatenate, axis=1)
dataframe.head()

,Title,Category,Price,Target,Description,Turn,Conversation
0,[Verizon Car Charger with Dual Output Micro US...,phone,10.0,"[7.0, 10.0]",[Charge two devices simultaneously on the go. ...,"[0, 1, 0, 1, 0, 1, 0, 1]","[Hi, not sure if the charger would work for my..."
1,"[Long board custom bilt from the board up, Lon...",bike,200.0,"[120.0, 200.0]",[Hey there cl I have a long board for sale pre...,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]","[Hi,I am interested in your board!, Hi, do you..."
2,[A 4 bedroom single family house for rent in c...,housing,3200.0,"[1920.0, 3200.0]",[This is a single family house in an excellent...,"[0, 1, 0, 1, 0, 1, 0, 1, 0]","[i would love to buy , Sure! What's your price..."
3,"[Sectional Couch, Sectional Couch]",furniture,585.0,"[444.0, 585.0]",[I am remodeling my space and selling a beauti...,"[0, 1, 0, 1, 0, 1]","[hello, Hi, How old is the couch?, It is pract..."
4,"[Beautiful Cherry Hutch, Beautiful Cherry Hutch]",furniture,1000.0,"[600.0, 1000.0]","[MUST SEE!!!,Like new ! - making space in our ...","[0, 1]","[500, that is a lot lower then i am willing to..."


In [13]:
# Define the function to check if the agent is a seller
def is_seller(data):
    return data['Turn'][0] == 1


dataframe['is_seller'] = dataframe.apply(is_seller, axis=1)
dataframe.head()

,Title,Category,Price,Target,Description,Turn,Conversation,is_seller
0,[Verizon Car Charger with Dual Output Micro US...,phone,10.0,"[7.0, 10.0]",[Charge two devices simultaneously on the go. ...,"[0, 1, 0, 1, 0, 1, 0, 1]","[Hi, not sure if the charger would work for my...",False
1,"[Long board custom bilt from the board up, Lon...",bike,200.0,"[120.0, 200.0]",[Hey there cl I have a long board for sale pre...,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]","[Hi,I am interested in your board!, Hi, do you...",False
2,[A 4 bedroom single family house for rent in c...,housing,3200.0,"[1920.0, 3200.0]",[This is a single family house in an excellent...,"[0, 1, 0, 1, 0, 1, 0, 1, 0]","[i would love to buy , Sure! What's your price...",False
3,"[Sectional Couch, Sectional Couch]",furniture,585.0,"[444.0, 585.0]",[I am remodeling my space and selling a beauti...,"[0, 1, 0, 1, 0, 1]","[hello, Hi, How old is the couch?, It is pract...",False
4,"[Beautiful Cherry Hutch, Beautiful Cherry Hutch]",furniture,1000.0,"[600.0, 1000.0]","[MUST SEE!!!,Like new ! - making space in our ...","[0, 1]","[500, that is a lot lower then i am willing to...",False


In [14]:
dataframe['is_seller'].value_counts()

False    2984
True     2263
Name: is_seller, dtype: int64

In [15]:
#print dataframe where is_seller is True
dataframe[dataframe['is_seller'] == True]

,Title,Category,Price,Target,Description,Turn,Conversation,is_seller
5,"[Office Client Waiting Room CHAIRS, Office Cli...",furniture,18.0,"[10.0, 18.0]","[Still sturdy and good, average condition. Som...","[1, 0, 1, 0, 1, 0]","[Hi, $15 sound good?, Can you come down a litt...",True
7,[Beautiful Renovated Studio with Balcony by La...,housing,1895.0,"[1743.0, 1895.0]","[This beautiful remodeled studio offers:,Refin...","[1, 0, 1, 0]","[Hello, I offer $500 for this studio., I accep...",True
8,"[2007 Mazda cx-9, 2007 Mazda cx-9]",car,8800.0,"[6688.0, 8800.0]",[2007 Mazda cx-9 2wheel drive. Truck has 138k ...,"[1, 0, 1]","[Good Days!, , ]",True
10,"[BLU R1 HD - 16 GB, BLU R1 HD - 16 GB]",phone,50.0,"[38.0, 50.0]",[BLU R1 HD - 16 GB - Black - Amazon Prime Excl...,"[1, 0]","[, Did you have the item for a while? ]",True
12,[Unlocked Brand New Apple iphone 7 Plus 128GB ...,phone,780.0,"[717.0, 780.0]","[We have rose gold, matte black and gold color...","[1, 0, 1, 0, 1, 0, 1, 0, 1]","[hi, Good afternoon, Brand new phone, still in...",True
...,...,...,...,...,...,...,...,...
5225,"[Antique Oak Accent Table, Antique Oak Accent ...",furniture,60.0,"[54.0, 60.0]",[This beautiful antique oak parlor table dates...,"[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]","[Hello. How are you?, Hi, I am interested in t...",True
5226,"[Pair of Mid Century Lamps, Pair of Mid Centur...",furniture,40.0,"[20.0, 40.0]",[Selling a pair of Mid Century lamps with orig...,"[1, 0, 1, 0, 1, 0, 1]","[HI! How are you?, Hi! Im very interested in ...",True
5230,[*** New Straight Talk Moto E 3G Android Prepa...,phone,60.0,"[30.0, 60.0]",[New Straight Talk Moto E 3G Android Prepaid S...,"[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]","[Hello there, Hi! Im wanting your Talk Moto 3G...",True
5234,"[Land Rover discovery SE series 2, Land Rover ...",car,3000.0,"[1500.0, 3000.0]",[For sale a 2001 Land Rover discovery series 2...,"[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]",[Hello! Are you interested in my wonderful lan...,True


In [34]:


# Define the function to convert data to a string prompt
def convert_data_to_prompt(data):
    # Extract conversation and item details
    conversation = data['Conversation']
    turns = data['Turn']
    list_of_prompts = []
    temp_df = pd.DataFrame(columns=['input', 'output'])
    # Initialize the prompt as an empty string
    input_prompt = """ """



    # Append item details to the prompt
    category = data['Category']
    price = data['Price']
    target = data['Target']
    description = data['Description'][0]
    title = data['Title'][0]
    

    input_prompt += f"Category: {category}, "
    input_prompt += f"Price: {price},"
    input_prompt += f"Target: {target}, "
    input_prompt += f"Title: {title}, "
    input_prompt += f"Description: {description}  "
    
    
    
    # Append conversation to the prompt
    for turn, dialogue in zip(turns, conversation):
        output_prompt = ""
        if turn == 0:
            input_prompt += f"buyer: {dialogue} "
            
        else:
            output_prompt += f"{dialogue} "
            
            temp_df = temp_df.append({'input': input_prompt, 'output': output_prompt}, ignore_index=True)
            input_prompt += f"seller: {dialogue} "

    return temp_df




In [35]:
x = convert_data_to_prompt(dataframe.iloc[99])

In [36]:
dataframe.iloc[99]['Conversation']

['None',
 "Hi. This apartment looks pretty good. Can we go lower than the advertised price? I'm thinking around 715 per month",
 "It's a car that I'm selling for 4400",
 'Interesting. My advertisement is for an apartment.  I can go with 4400 for the car. ',
 '']

In [37]:
dataframe.iloc[99]['Turn']

[1, 0, 1, 0, 1]

In [38]:
x

,input,output
0,"Category: housing, Price: 946.0,Target: [718....",None
1,"Category: housing, Price: 946.0,Target: [718....",It's a car that I'm selling for 4400
2,"Category: housing, Price: 946.0,Target: [718....",


In [39]:
final_df = pd.DataFrame(columns=['input', 'output'])

In [40]:
for index in range(dataframe.shape[0]):
    final_df = final_df.append(convert_data_to_prompt(dataframe.iloc[index]), ignore_index=True)

final_df

,input,output
0,"Category: phone, Price: 10.0,Target: [7.0, 10...","It will work, i have never seen a car without ..."
1,"Category: phone, Price: 10.0,Target: [7.0, 10...",I think the lowest I would want to go is 8.
2,"Category: phone, Price: 10.0,Target: [7.0, 10...","7, and we have a deal."
3,"Category: phone, Price: 10.0,Target: [7.0, 10...",
4,"Category: bike, Price: 200.0,Target: [120.0, ...","Hi, do you have any questions?"
...,...,...
21545,"Category: housing, Price: 2025.0,Target: [101...","2 bed, one bath studio"
21546,"Category: housing, Price: 2025.0,Target: [101...","The listing price is $2025, and I was told by ..."
21547,"Category: housing, Price: 2025.0,Target: [101...","Yes, you can use the other room as a bedroom. ..."
21548,"Category: housing, Price: 2025.0,Target: [101...",1800 and we'll have a deal. I could give you a...


In [53]:
#check empty rows in output column
final_df[final_df['output']==' ' ].shape



(2501, 2)

In [54]:
#remove rows contains " "
final_df = final_df[final_df['output'] != ' ']

In [55]:
final_df

,input,output
0,"Category: phone, Price: 10.0,Target: [7.0, 10...","It will work, i have never seen a car without ..."
1,"Category: phone, Price: 10.0,Target: [7.0, 10...",I think the lowest I would want to go is 8.
2,"Category: phone, Price: 10.0,Target: [7.0, 10...","7, and we have a deal."
4,"Category: bike, Price: 200.0,Target: [120.0, ...","Hi, do you have any questions?"
5,"Category: bike, Price: 200.0,Target: [120.0, ...",I do not know specifically but the brand is a ...
...,...,...
21544,"Category: housing, Price: 2025.0,Target: [101...",Hi there. Are you interested in the place?
21545,"Category: housing, Price: 2025.0,Target: [101...","2 bed, one bath studio"
21546,"Category: housing, Price: 2025.0,Target: [101...","The listing price is $2025, and I was told by ..."
21547,"Category: housing, Price: 2025.0,Target: [101...","Yes, you can use the other room as a bedroom. ..."


In [56]:
final_df.to_csv('prompts.csv')